In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.geocoders import Nominatim
from sklearn.preprocessing import StandardScaler



pd.set_option('display.max_columns', 100)

In [23]:
#loading dataframes
df1=pd.read_excel("Telco_customer_churn.xlsx")
df2=pd.read_excel("Telco_customer_churn_status.xlsx")
df3=pd.read_excel("Telco_customer_churn_services.xlsx")
df4=pd.read_excel("Telco_customer_churn_demographics.xlsx")

In [24]:
#Mergin dataframes into one
df1_2=pd.merge(df1, df2, left_on='CustomerID', right_on='Customer ID', how='left').drop('Customer ID', axis=1)
df3_4=pd.merge(df3, df4, left_on='Customer ID', right_on='Customer ID', how='left')
customer_df=pd.merge(df1_2, df3_4, left_on='CustomerID', right_on='Customer ID', how='left').drop('Customer ID', axis=1)

In [25]:
#Cleaning up duplicates 
customer_df=customer_df.T.drop_duplicates().T
customer_df.columns=[column.lower().strip("_x").strip("_y").replace(" ","_")for column in customer_df.columns]  
customer_df["internet_services"]=customer_df["internet_service"].iloc[:,1:2]
customer_df["online_security"]=customer_df["online_securit"].iloc[:,1:2]
customer_df["online_backup_srvc"]=customer_df["online_backup"].iloc[:,1:2]
customer_df["multiple_line"]=customer_df["multiple_lines"].iloc[:,1:2]
customer_df["streaming_tv_srvc"]=customer_df["streaming_tv"].iloc[:,1:2]
customer_df["payment_methods"]=customer_df["payment_method"].iloc[:,:1]
customer_df["total_charge"]=customer_df["total_charges"].iloc[:,:1]
customer_df["contracts"]=customer_df["contract"].iloc[:,1:2]
customer_df["churn_reasons"]=customer_df["churn_reason"].iloc[:,1:2]
customer_df["streaming_movies_srvc"]=customer_df["streaming_movies"].iloc[:,1:2]

In [26]:
#Dropping Redundant columns
customer_df.drop(["total_charges","churn_score","churn_reason","lat_long","customerid","payment_method","contract","churn_label","count","countr","state","quarter","internet_service","online_securit","online_backup","multiple_lines","streaming_tv","streaming_movies"],axis=1,inplace=True)

In [27]:
#Cleaning missing values from "total_charge" column
hold=[]
for x in customer_df.index:
    if type(customer_df.total_charge[x])== str:
        hold.append(0)
    else:
        hold.append(customer_df.total_charge[x])
customer_df.total_charge=hold

In [28]:
#Binning age 
customer_df.age=pd.cut(x=customer_df['age'], bins=[10,20,30,40, 50,60,70,80], labels=["10","20","30","40", "50","60","70"])

In [29]:
#Filling NA values for churn reason
customer_df.churn_reasons=customer_df.churn_reasons.fillna("No reason given")

Filling the column of Churn Category, by comparing if the customer churned or not. If the customer didnt churn, Fill it with "Not Churn" to add the category that the customer did not leave, therefore filling the missing valuees. If the customer left, then fill it with "No reason" since we dont have a reason of why they left.

In [30]:
conditions=[
    (customer_df["churn_categor"].isna())& (customer_df["churn_value"]==0),            
    (customer_df["churn_categor"].isna())& (customer_df["churn_value"]==1),
]
choises=[
    "Not Churned",
    "No Reason",

]
customer_df.churn_categor=np.select(conditions,choises,default=customer_df.churn_categor)

In [31]:
 def to_binary(column):
    """
    Takes in a column and turns its value from a Yes or No to a binary. 
    """
    conditions=[
        (customer_df[column].str.lower()=='yes'),            
        (customer_df[column].str.lower()=='no'),
    ]
    choises=[
        1,
        0,

    ]
    customer_df[column]=np.select(conditions,choises,default=0)

In [32]:
#converting columns into binary
col_to_bin=["senior_citizen","device_protection_plan","partner","dependents","phone_service","paperless_billing","referred_a_friend","premium_tech_support","streaming_music","unlimited_data","under_30","online_security","multiple_line","online_backup_srvc","streaming_tv_srvc","streaming_movies_srvc","internet_services","tech_support","device_protection"]
for x in col_to_bin:
    to_binary(x)

In [33]:
customer_df

,cit,zip_code,latitude,longitude,gender,senior_citizen,partner,dependents,tenure_months,phone_service,device_protection,tech_support,paperless_billing,monthly_charges,churn_value,cltv,satisfaction_score,customer_status,churn_categor,referred_a_friend,number_of_referrals,tenure_in_months,offer,avg_monthly_long_distance_charges,internet_type,avg_monthly_gb_download,device_protection_plan,premium_tech_support,streaming_music,unlimited_data,total_refunds,total_extra_data_charges,total_long_distance_charges,total_revenue,age,under_30,number_of_dependents,internet_services,online_security,online_backup_srvc,multiple_line,streaming_tv_srvc,payment_methods,total_charge,contracts,churn_reasons,streaming_movies_srvc
0,Los Angeles,90003,33.9641,-118.273,Male,0,0,0,2,1,0,0,1,53.85,1,3239,1,Churned,Competitor,0,0,2,None,10.47,DSL,21,0,0,0,1,0,0,20.94,129.09,30,0,0,1,1,1,0,0,Mailed check,108.15,Month-to-Month,Competitor made better offer,0
1,Los Angeles,90005,34.0593,-118.307,Female,0,0,1,2,1,0,0,1,70.7,1,2701,2,Churned,Other,0,0,2,None,9.12,Fiber Optic,51,0,0,0,1,0,0,18.24,169.89,10,1,2,1,0,0,0,0,Electronic check,151.65,Month-to-Month,Moved,0
2,Los Angeles,90006,34.048,-118.294,Female,0,0,1,8,1,1,0,1,99.65,1,5372,3,Churned,Other,0,0,8,None,12.15,Cable,26,1,0,1,1,0,0,97.2,917.7,30,0,2,1,0,0,1,1,Electronic check,820.50,Month-to-Month,Moved,1
3,Los Angeles,90010,34.0621,-118.316,Female,0,1,1,28,1,1,1,1,104.8,1,5003,3,Churned,Other,0,0,28,Offer C,4.89,Fiber Optic,47,1,1,1,1,0,0,136.92,3182.97,20,1,3,1,0,0,1,1,Electronic check,3046.05,Month-to-Month,Moved,1
4,Los Angeles,90015,34.0392,-118.266,Male,0,0,1,49,1,1,0,1,103.7,1,5340,1,Churned,Competitor,0,0,49,None,44.33,Fiber Optic,11,1,0,1,1,0,0,2172.17,7208.47,30,0,1,1,0,1,1,1,Bank transfer (automatic),5036.30,Month-to-Month,Competitor had better devices,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Landers,92285,34.3417,-116.539,Female,0,0,0,72,1,0,0,1,21.15,0,5306,5,Stayed,Not Churned,0,0,72,None,22.77,None,0,0,0,0,0,19.31,0,1639.44,3039.53,20,0,0,0,0,0,0,0,Bank transfer (automatic),1419.40,Two Year,No reason given,0
7039,Adelanto,92301,34.6678,-117.536,Male,0,1,1,24,1,1,1,1,84.8,0,2140,3,Stayed,Not Churned,1,1,24,Offer C,36.05,Cable,24,1,1,1,1,48.23,0,865.2,2807.47,30,0,2,1,1,0,1,1,Mailed check,1990.50,One Year,No reason given,1
7040,Amboy,92304,34.5599,-115.637,Female,0,1,1,72,1,1,0,1,103.2,0,5560,4,Stayed,Not Churned,1,4,72,None,29.66,Fiber Optic,59,1,0,1,1,45.38,0,2135.52,9453.04,20,0,2,1,0,1,1,1,Credit card (automatic),7362.90,One Year,No reason given,1
7041,Angelus Oaks,92305,34.1678,-116.864,Female,0,1,1,11,0,0,0,1,29.6,0,2793,4,Stayed,Not Churned,1,1,11,None,0,DSL,17,0,0,0,1,27.24,0,0,319.21,30,0,2,1,1,0,0,0,Electronic check,346.45,Month-to-Month,No reason given,0


In [12]:
customer_df.to_csv("clean_customer_df.csv")

####  Preparing Dataframe for modeling. 

In [13]:
# columns_a=["latitude","longitude","tenure_months","monthly_charges","cltv","tenure_in_months","avg_monthly_long_distance_charges","avg_monthly_gb_download","total_refunds","total_long_distance_charges","total_revenue","total_charge"]
# columns_b=[x for x in customer_df.columns if x not in columns_a ]
# dfa=customer_df[columns_a].copy()
# dfb=customer_df[columns_b].copy()
# scaler = StandardScaler()
# dfa=pd.DataFrame(data=scaler.fit_transform(dfa),columns=dfa.columns)
# customer_df=pd.concat([dfa,dfb],axis=1)

In [14]:
customer_df=pd.get_dummies(customer_df,columns=["cit","customer_status","churn_categor","age","zip_code","gender","churn_reasons","offer","internet_type","contracts","payment_methods","number_of_dependents"],drop_first=True)


In [15]:
customer_df=customer_df.astype("float")

In [16]:
customer_df.to_csv("modeling_customer_df.csv")

# Vanilla Model

In [17]:
y=customer_df["churn_value"]
X=customer_df[customer_df.columns[customer_df.columns!="churn_value"]]

In [18]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=2020, test_size=0.2)

In [19]:
# Fitting our model
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression(class_weight='balanced')
lr.fit(X_train,y_train)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(class_weight='balanced')

In [20]:
y_pred_lr = lr.predict(X_test)

In [21]:
from sklearn import metrics
print('Logistic Regression Accuracy: {}'.format(metrics.accuracy_score(y_test, y_pred_lr)))
print('Logistic Regression F1: {}'.format(metrics.f1_score(y_test,y_pred_lr)))

Logistic Regression Accuracy: 0.7395315826827538
Logistic Regression F1: 0.6032432432432432
